In [1]:
import re
import statistics as stat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scrapping_with_selenium as sc  # scrapping_with_selenium: past project which scrapes data from wuzzuf(check it in my past repo)

In [2]:
# the function which scrapes the data for the firt 150

table = sc.scraper("Data", 150, ["Analyst", "Data Engineer", "Data Scientist", "Bi", "Business Intelligence","Data Entry"])

In [3]:
table.to_csv("D:\Projects\Data Roles Market\df.csv", index = False)

In [5]:
# read the csv file 
df = pd.read_csv("D:\Projects\Data Roles Market\df.csv")
print(df.skill.unique)

<bound method Series.unique of 0                        cloudera
1            Computer Engineering
2                            Data
3                   Data Modeling
4                        Modeling
5                           Spark
6                Computer Science
7                   Data Engineer
8                            Data
9                        engineer
10    Information Technology (IT)
11                    Engineering
12           Software Development
Name: skill, dtype: object>


In [ ]:
# only keep job titles that contains(Data, Analyst & Bi)
df= df[
df["job_title"].str.contains("Data") |
df["job_title"].str.contains("Analyst") |
df["job_title"].str.contains("BI")
]


In [ ]:
# create a list of our conditions
conditions = [
df["job_title"].str.contains("entry"),
df["job_title"].str.contains("Entry"),
df["job_title"].str.contains("Analyst"),
df["job_title"].str.contains("BI"),
df["job_title"].str.contains("Engineer"),
df["job_title"].str.contains("Scientist")
]

# create a list of the values we want to assign for each condition
values = ['data entry', 'data entry', 'data analyst', 'bi developer','data engineer', 'data scientist']

# create a new column and use np.select to assign values to it using our lists as arguments
df['roles'] = np.select(conditions, values)

In [ ]:
df[["city","state","country"]] = df['location'].str.split(',', expand=True)

In [ ]:
df["company"] = df["company"].str.replace(" -", "")
df["experience"] = df["experience"].str.replace(" ", "")
df["experience"] = df["experience"].str.findall(".(.+?)Y")
df["experience"] = [''.join(map(str, l)) for l in df["experience"]]

In [ ]:
conditions = [
df["experience"].str.contains("^0.*"),
df["experience"].str.contains("^1.*"),
df["experience"].str.contains("^[23].*"),
df["experience"].str.contains("^[45].*"),
df["experience"].str.contains("^[67].*")
]

values = ['entry level', 'junior', 'mid level', 'senior','team lead']

df['levels'] = np.select(conditions, values)

In [ ]:
def clean_salaries(df, rol, lst):
        
        conds=[
        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'entry level'),

        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'junior'),

        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'mid level'),

        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'senior'),

        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'team lead')
        ]

        for _ in range(len(conds)):
                df['salaries'] = np.where(conds[_], lst[_], df["salaries"])


In [ ]:
# replace confidential with avg market salaries lists
sal = {
    "data entry":[4000, 4000, 4000, 5400, 5400],
    "data analyst":[6000, 8000, 10950, 28500, 28500],
    "bi developer":[5625, 8550, 10000, 15000, 20000],
    "data engineer":[9000, 10375, 23000, 30000, 47000],
    "data scientist":[9400, 11000, 19200, 39584, 67815]
}

for i in sal:
    clean_salaries(df, i, sal[i])        


In [ ]:
# replace ("number" To "number") expression, to only mean value
df["salaries"] = df["salaries"].apply(lambda salary: re.findall(("\d+"), salary) if type(salary) == str else salary)
df["salaries"] = df["salaries"].apply(lambda salary: list(map(int, salary)) if type(salary) == list else salary)
df["salaries"] = df["salaries"].apply(lambda salary: stat.mean(salary) if type(salary) == list else salary)

#manually convert usd to egp 
df.loc[39,"salaries"] = 8250
